In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils import data
from d2l import torch as d2l
from tqdm import tqdm

In [2]:
df_map = pd.read_csv('./per_job_map.csv')
df_map

,job,person,label
0,1135,40121,0
1,305,4973,0
2,664,9112,0
3,221,13978,0
4,322,41717,0
...,...,...,...
35286,1213,2949,0
35287,284,19247,0
35288,748,39496,0
35289,491,7187,1


In [41]:
# 读取属性嵌入
def get_info(path: str):
  info = pd.read_csv(path,sep=' ',header=None,na_values=[])
  info.drop(columns=[0,info.shape[1]-1],inplace=True)
  return info

info_job = get_info('./embedding_recruit.txt')
info_per = get_info('./embedding_person.txt')
info_job.shape, info_per.shape

((2104, 64), (54779, 219))

In [33]:
# 读取结构嵌入
def get_dest(path: str):
  dest = pd.read_csv(path,sep=' ',header=None)
  dest = dest.sort_values(0)
  dest.index = np.arange(dest.shape[0])
  return dest.drop(columns=0)
  
dest_job = get_dest('./destruct_job.txt')
dest_per = get_dest('./destruct_person.txt')
dest_job.shape, dest_per.shape

((2104, 128), (54779, 128))

In [43]:
# 拼接属性和结构，共539维
df_job_all = pd.concat([info_job, dest_job], axis=1)
df_per_all = pd.concat([info_per, dest_per], axis=1)
df_job_all.shape, df_per_all.shape

((2104, 192), (54779, 347))

In [45]:
def get_input(job_id, per_id):
  job = df_job_all.iloc[job_id-1].values
  per = df_per_all.iloc[per_id-1].values
  return np.concatenate([job, per],axis=0)

In [46]:
all_x = []
all_y = []
for i in tqdm(df_map.index):
  row = df_map.iloc[i]
  feature = get_input(row['job'],row['person'])
  if row['label']==0:
    label = [1,0.5]
  elif row['label']==1:
    label = [1,1]
  else:
    label = [0,0]
  label = np.array(label)
  all_x.append(feature)
  all_y.append(label)

100%|██████████| 35291/35291 [00:18<00:00, 1906.36it/s]


In [53]:
all_x = torch.Tensor(np.array(all_x))
all_y = torch.Tensor(np.array(all_y))
all_x.shape, all_y.shape

(torch.Size([35291, 539]), torch.Size([35291, 2]))

In [ ]:
batch_size = 50
train_size = 30000
test_size = 1000

In [58]:
net = nn.Sequential(nn.Linear(539, 128),
                    nn.ReLU(),
                    nn.Linear(128, 2))
net

Sequential(
  (0): Linear(in_features=539, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=2, bias=True)
)

In [59]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
  # Array to Iter
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [57]:
lr, num_epochs = 0.01, 50
loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.SGD(net.parameters(), lr=lr)

In [5]:
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [10]:
net(next(iter(test_iter)))

AttributeError: 'list' object has no attribute 'flatten'